In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
import base64
from animalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "1234"
shelter = AnimalShelter(username, password)



# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))
# print(df)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('Animal Shelter Dashboard - by Debynyhan'))),
    html.Center(html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                               style={'max-width': '250px', 'max-height': '150', 'width': 'auto', 'height': 'auto'}), href='https://www.snhu.edu')),
    html.Hr(),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water', 'value': 'water'},
            {'label': 'Mountain or Widerness', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'reset'},
        ],
        value='reset'

    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,

    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
# @app.callback([Output('datatable-id','data'),
#                Output('datatable-id','columns')],
#               [Input('filter-type', 'value')])
# def update_dashboard(filter_type):
    
#     if filter_type  == 'water':
#         df_filtered = df[(df['animal_type'] == 'Dog') & 
#                         (df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) &
#                         (df['sex_upon_outcome'] == 'Intact Female') &
#                         (df['age_upon_outcome_in_weeks'] >= 26) &
#                         (df['age_upon_outcome_in_weeks'] <= 156)]
#     elif filter_type  == 'mountain':
#         df_filtered = df[(df['animal_type'] == 'Dog') & 
#                         (df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) &
#                         (df['sex_upon_outcome'] == 'Intact Female') &
#                         (df['age_upon_outcome_in_weeks'] >= 26) &
#                         (df['age_upon_outcome_in_weeks'] <= 156)]  
#     elif filter_type  == 'disaster':
#         df_filtered = df[(df['animal_type'] == 'Dog') & 
#                         (df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) &
#                         (df['sex_upon_outcome'] == 'Intact Female') &
#                         (df['age_upon_outcome_in_weeks'] >= 26) &
#                         (df['age_upon_outcome_in_weeks'] <= 156)]    
#         else:
#             df_filtered = df
#         return df_filtered.to_dict('record')    
### Add code to filter interactive data table with MongoDB queries
     ###  Add code to filter interactive data table with MongoDB queries
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])  
def update_dashboard(filter_type):    
    if filter_type == 'water':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "NewfoundLand"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }  
    elif filter_type == 'mountain':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",  "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }     
    elif filter_type == 'disaster':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        } 
    else: # reset option, show all records
        query = {}
        
    data = pd.DataFrame.from_records(shelter.read((query)))
                                     
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in data.columns]
                                     
                                     
    data_dict=data.to_dict('records')
        
        
    return data_dict, columns




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    df_viewData = pd.DataFrame(viewData)
    pie_chart = px.pie(df_viewData, names='breed')
    return [
       dcc.Graph(            
           figure = pie_chart
       )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    
    #Filter Dataframe to only include dogs
    dff = dff[dff['animal_type'] == 'Dog'] 
    # Austin TX is  [30.75, =97.48]
    return [
        dl.Map(style={'width':  ' 1000px', 'height':'500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tooltip and popup
            dl.Marker(position=[dff.iloc[0]['location_lat'], dff.iloc[0]['location_long'] ], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1('Animal ID: ' + str(dff.iloc[0]['animal_id'])),
                    html.P('Latitude: ' + str(dff.iloc[0]['location_lat'])),
                    html.P('longitude: ' + str(dff.iloc[0]['location_long'])),
                ])
            ])
        ])
    ]




app

Connection was successful
